In [ ]:
# -------------------------------------
# 1. Setup (Colab has TF pre-installed)
# -------------------------------------
import tensorflow as tf
from tensorflow.keras import layers, models

print("TensorFlow version:", tf.__version__)

# -------------------------------------
# 2. Load a sample dataset (replace with your own)
# Colab provides tf.keras.datasets, here we use CIFAR-10 for demo
# -------------------------------------
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.cifar10.load_data()

# Normalize images
x_train = x_train.astype("float32") / 255.0
x_val = x_val.astype("float32") / 255.0

# Resize images to 224x224 (needed for mobilenet/MobileNet)
resize_layer = tf.keras.Sequential([
    layers.Resizing(224, 224)
])

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).map(lambda x, y: (resize_layer(x), y))
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(32).map(lambda x, y: (resize_layer(x), y))

# -------------------------------------
# 3. Transfer Learning with mobilenet50
# -------------------------------------
base_mobilenet = tf.keras.applications.MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_mobilenet.trainable = False  # freeze backbone
mobilenet_model = models.Sequential([
    base_mobilenet,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(10, activation="softmax")  # CIFAR-10 has 10 classes
])

mobilenet_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

print("Training mobilenet50...")
mobilenet_model.fit(train_ds, validation_data=val_ds, epochs=3)

# -------------------------------------
# 4. Fine-tuning (optional: unfreeze last few layers)
# -------------------------------------
base_mobilenet.trainable = True
for layer in base_mobilenet.layers[:-30]:
    layer.trainable = False

mobilenet_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # lower LR
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

print("Fine-tuning mobilenet50...")
mobilenet_model.fit(train_ds, validation_data=val_ds, epochs=2)

TensorFlow version: 2.19.0
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Training mobilenet50...
Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 94s 48ms/step - accuracy: 0.5427 - loss: 1.3591 - val_accuracy: 0.7737 - val_loss: 0.6552
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 57s 36ms/step - accuracy: 0.7523 - loss: 0.7207 - val_accuracy: 0.7943 - val_loss: 0.5886
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 82s 36ms/step - accuracy: 0.7804 - loss: 0.6378 - val_accuracy: 0.8022 - val_loss: 0.5571
Fine-tuning mobilenet50...
Epoch 1/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 100s 53ms/step - accuracy: 0.7143 - loss: 0.8375 - val_accuracy: 0.8226 - val_loss: 0.5110
Epoch 2/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 69s 44ms/step - accuracy: 0.8162 - loss: 0.5374 - val_accuracy: 0.8415 - val_loss: 0.4547


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/cat.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0) # batch dimension

pred_probs = mobilenet_model.predict(img_array)
pred_class = np.argmax(pred_probs, axis=1)[0]

print("Predicted class:", pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
Predicted class: 3
